In [1]:
import numpy as np
import math
import os
import pickle
import pandas as pd
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt

def _rmse (x, x_hat):
    x = x[1:]
    x_hat = x_hat[1:]
    return np.sqrt(np.mean(((x-x_hat)**2)))

## Load Data Sets in Memory

In [2]:
path = 'Path to the Data'
with open(path+'train_resampled.pickle', 'rb') as handle:
    train_resampled = pickle.load(handle)
with open(path+'test_resampled.pickle', 'rb') as handle:
    test_resampled = pickle.load(handle)
with open(path+'dis_train.pickle', 'rb') as handle:
    dis_train = pickle.load(handle)
with open(path+'dis_test.pickle', 'rb') as handle:
    dis_test = pickle.load(handle)
with open(path+'Scalar.pickle', 'rb') as handle:
    Scalar = pickle.load(handle)
train_resampled = train_resampled[:13400]
dis_train = dis_train[:13400]
train_resampled = [train_resampled[idx][:-1].values for idx in range(len(train_resampled))]
test_resampled = [test_resampled[idx].values[:-1] for idx in range(len(test_resampled))]

## Variational Recurrent Neural Network (VRNN-I-S)

In [7]:
class VRNN(nn.Module):
    
    def __init__(self, x_dim, h_dim, z_dim, n_layers, bias = True ):
        
        super(VRNN, self).__init__()

        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim
        self.n_layers = n_layers

        #feature-extracting transformations
        self.phi_x = nn.Sequential( nn.Linear(self.x_dim , self.h_dim), nn.Tanh(), 
                                   nn.Linear(self.h_dim, self.h_dim), nn.Tanh())
        self.phi_z = nn.Sequential( nn.Linear(self.z_dim, self.h_dim), nn.Tanh())

        #encoder
        self.enc = nn.Sequential( nn.Linear(self.h_dim + self.h_dim * self.n_layers , 
                                            self.h_dim), nn.Tanh(), nn.Linear(self.h_dim, self.h_dim), nn.Tanh())
        
        self.enc_mean = nn.Sequential( nn.Linear(self.h_dim, self.z_dim), nn.Tanh())
        self.enc_std = nn.Sequential( nn.Linear(self.h_dim, self.z_dim), nn.Sigmoid())

        
        #decoder
        self.dec = nn.Sequential( nn.Linear(self.h_dim + self.h_dim * self.n_layers , self.h_dim), nn.Tanh(), 
                                 nn.Linear(self.h_dim, self.h_dim), nn.Tanh())
        self.dec_std = nn.Sequential( nn.Linear(self.h_dim, self.x_dim), nn.Sigmoid())
        #self.dec_mean = nn.Linear(h_dim, x_dim)
        self.dec_mean = nn.Sequential( nn.Linear( self.h_dim, self.x_dim ), nn.Tanh())

        #recurrence
        self.rnn = nn.GRU ( self.h_dim + self.h_dim + self.n_layers * self.h_dim, self.h_dim, 
                           self.n_layers, bias , batch_first = True )
        
    def encode (self, phi_x_t , h):
        phi_x_t = self._concatenate (phi_x_t , h)
        enc_t = self.enc(phi_x_t)
        enc_mean_t = self.enc_mean(enc_t)
        enc_std_t = self.enc_std(enc_t)
        return enc_mean_t , enc_std_t
        
    
    def decode (self, phi_z_t , h):
        phi_z_t = self._concatenate (phi_z_t , h)
        dec_t = self.dec(phi_z_t)
        dec_mean_t = self.dec_mean(dec_t)
        dec_std_t = self.dec_std(dec_t)
        return dec_mean_t , dec_std_t
        
    def forward(self, x):
        all_enc_mean, all_enc_std = [], []
        all_dec_mean, all_dec_std = [], []
        kld_loss = 0
        nll_loss = 0

        h = Variable(torch.zeros(self.n_layers, x.size(0), self.h_dim))
        
        for t in range(x.size(1)):
            
            phi_x_t = self.phi_x(x[:,t,:].float())
            #encoder
            enc_mean_t , enc_std_t = self.encode(phi_x_t , h)
            #sampling and reparameterization
            z_t = self._reparameterized_sample(enc_mean_t, enc_std_t)
            phi_z_t = self.phi_z(z_t)
            #decoder
            dec_mean_t , dec_std_t = self.decode(phi_z_t , h)
            
            #recurrence
            temp = self._concatenate ( torch.cat ( [ phi_x_t , phi_z_t ] , 1) , h )
            _, h = self.rnn(temp.reshape(temp.shape[0] , 1 , temp.shape[1]))

            #computing losses enc_std_t.mul(0.5).exp_()
            #kld_loss += self._kld_gauss(enc_mean_t, enc_std_t, prior_mean_t, prior_std_t)
            kld_loss += self._kld_gauss(enc_mean_t, enc_std_t)
            nll_loss += self._nll_gauss(dec_mean_t, dec_std_t, x[:,t,:])
                
            all_enc_std.append(enc_std_t)
            all_enc_mean.append(enc_mean_t)
            all_dec_mean.append(dec_mean_t)
            all_dec_std.append(dec_std_t)


        return kld_loss, nll_loss,(all_enc_mean, all_enc_std),(all_dec_mean, all_dec_std)
    
    def _reparameterized_sample(self, mean, logvar):
        """using std to sample"""
        std = logvar.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mean)


    def _kld_gauss(self, mu, logvar):
        """Using std to compute KLD"""
        return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        

    def _nll_gauss(self, mean, logvar , x):
        return torch.sum( 0.5 * np.log (2 * np.pi) + 0.5 * logvar + (x-mean)**2 / (2 *  torch.exp(logvar)) )
    
    def _concatenate (self , a , b) :
        for i in range(len(b)):
            a = torch.cat([a , b[i]] , 1)
        return a

## Configure the Hyper-Parameters and Model

In [8]:
#hyperparameters
x_dim = 25
h_dim = 50
z_dim = 2
n_layers =  2
n_epochs = 5
clip = 10
learning_rate = 1e-3
batch_size = 100
seed = 100
print_every = 10
save_every = 10
#manual seed
torch.manual_seed(seed)
#init model + optimizer + datasets
train_loader = torch.utils.data.DataLoader ( dataset = train_resampled ,  batch_size = 100 , shuffle= True)
test_loader = torch.utils.data.DataLoader (  dataset = test_resampled , shuffle= True)
model = VRNN(x_dim, h_dim, z_dim, n_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train and Test the Model

In [9]:
def train(epoch):
    train_loss = 0
    epoch_loss = np.zeros(int(len (train_resampled) / batch_size ))
    epoch_div = np.zeros(int(len (train_resampled) / batch_size))
    for batch_idx, (data) in enumerate(train_loader):
        
        data = Variable(data)
        #forward + backward + optimize
        optimizer.zero_grad()
        kld_loss, nll_loss, _, _ = model(data)
        epoch_loss [batch_idx] = nll_loss
        epoch_div [batch_idx] = kld_loss
        loss = kld_loss + nll_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        #printing
        if batch_idx % print_every == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t KLD Loss: {:.6f} \t NLL Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                kld_loss.data / batch_size,
                nll_loss.data / batch_size))

            

        train_loss += loss.data
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    return epoch_loss, epoch_div
    
def test(epoch):
    """uses test data to evaluate 
    likelihood of the model"""
    mean_kld_loss, mean_nll_loss = 0, 0
    epoch_loss = np.zeros(len(test_resampled))
    epoch_div = np.zeros(len(test_resampled))
    for i, (data) in enumerate(test_loader):                                           
        
        data = Variable(data.reshape(1,47,25))
        kld_loss, nll_loss, _, _ = model(data)
        epoch_div [i] = kld_loss
        epoch_loss [i] = nll_loss
        mean_kld_loss += kld_loss.data
        mean_nll_loss += nll_loss.data

    mean_kld_loss /= len(test_loader.dataset)
    mean_nll_loss /= len(test_loader.dataset)

    print('====> Test set loss: KLD Loss = {:.4f}, NLL Loss = {:.4f} '.format(
        mean_kld_loss, mean_nll_loss))
    return epoch_loss, epoch_div

## Run the Training and Validation/Testing

In [10]:
train_error = np.zeros([n_epochs , int(len (train_resampled) / batch_size ) ])
train_div = np.zeros([n_epochs , int(len (train_resampled) / batch_size ) ])
test_error , test_div  = np.zeros([n_epochs , len(test_resampled)]) , np.zeros([n_epochs , len(test_resampled)]) 
for epoch in range(1, n_epochs + 1):
    #training + testing
    tr = train(epoch)
    train_error [epoch-1 , :] = tr [0]
    train_div [epoch-1 , :] = tr [1] 
    te = test(epoch)
    test_error [epoch-1 , :] = te [0]
    test_div [epoch-1 , :] = te [1]

Train Epoch: 1 [0/13400 (0%)]	 KLD Loss: 7.276603 	 NLL Loss: 1703.674400
Train Epoch: 1 [1000/13400 (7%)]	 KLD Loss: 2.734653 	 NLL Loss: 1524.999991
Train Epoch: 1 [2000/13400 (15%)]	 KLD Loss: 0.463646 	 NLL Loss: 1331.308182
Train Epoch: 1 [3000/13400 (22%)]	 KLD Loss: 0.155726 	 NLL Loss: 1228.470475
Train Epoch: 1 [4000/13400 (30%)]	 KLD Loss: 0.117668 	 NLL Loss: 1166.655844
Train Epoch: 1 [5000/13400 (37%)]	 KLD Loss: 0.076995 	 NLL Loss: 1130.281179
Train Epoch: 1 [6000/13400 (45%)]	 KLD Loss: 0.036912 	 NLL Loss: 1110.422016
Train Epoch: 1 [7000/13400 (52%)]	 KLD Loss: 0.021121 	 NLL Loss: 1099.923806
Train Epoch: 1 [8000/13400 (60%)]	 KLD Loss: 0.009476 	 NLL Loss: 1094.306625
Train Epoch: 1 [9000/13400 (67%)]	 KLD Loss: 0.006231 	 NLL Loss: 1091.324335
Train Epoch: 1 [10000/13400 (75%)]	 KLD Loss: 0.004099 	 NLL Loss: 1089.686865
Train Epoch: 1 [11000/13400 (82%)]	 KLD Loss: 0.003149 	 NLL Loss: 1088.753556
Train Epoch: 1 [12000/13400 (90%)]	 KLD Loss: 0.001663 	 NLL Loss: 

## Save the Training Data, Model, Error

In [8]:
with open(path +'P_C_train_error.pickle', 'wb') as handle:
    pickle.dump(train_error, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path +'P_C_test_error.pickle', 'wb') as handle:
    pickle.dump(test_error, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path +'P_C_train_div.pickle', 'wb') as handle:
    pickle.dump(train_div, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path +'P_C_test_div.pickle', 'wb') as handle:
    pickle.dump(test_div, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path +'P_C_Model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path +'P_C_train_loader.pickle', 'wb') as handle:
    pickle.dump(train_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(path +'P_C_test_loader.pickle', 'wb') as handle:
    pickle.dump(test_loader, handle, protocol=pickle.HIGHEST_PROTOCOL)

## RMSE on Test Data Set

In [11]:
path = 'Path to the Test Resampled Time Series'
with open(path, 'rb') as handle:
    test_resampled = pickle.load(handle)
test_resampled = [test_resampled[idx].values for idx in range(len(test_resampled))]
result =  [ model (torch.tensor (test_resampled[idx]).reshape(1,48,25)) for idx in range(len(test_resampled)) ]
torch.manual_seed(seed)
dist = [ torch.distributions.normal.Normal (result[idx][3][0][0][:7] , result[idx][3][1][0][:7].mul(0.5).exp_()) for idx in range(len(result))]
recon_x = [ dist[idx].sample((10000,)).mean(0) for idx in range(len(dist)) ]
error = [ _rmse(test_resampled[idx][-1][:7] , np.array(recon_x[idx][-1][:7])) for idx in range(len(test_resampled)) ]
print ('Average RMSE for Test Data is: '+str(np.mean(error)))
print ('Variance of RMSE for Test Data is: '+str(np.std(error)))

Average RMSE for Test Data is: 0.010347915070650334
Variance of RMSE for Test Data is: 0.003067595306531375
